In [1]:
import re
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from nltk import stem  

In [2]:
def re_rule(x):
    if re.search("[a-z0-9]+",x) != None :
        return re.search("[a-z0-9]+",x).group(0)

In [3]:
stop_words = set(stopwords.words('english'))
stop_words.update(['.',',','"',"'",'?','!',':',';','(',')','[rm]','[',']','{','}'])
wordnet_lemm = stem.WordNetLemmatizer()

def NLP_process(Sentence):
    try:
        words_result = []
        words_now = word_tokenize(Sentence)
        words_result = words_now
        words_now = [i.lower() for i in words_result if i.lower() not in stop_words]
        words_result = words_now
        words_now =  [re_rule(w) for w in words_result]
        words_result = []
        for w in words_now:
            if w != None :
                words_result.append(
                    wordnet_lemm.lemmatize(w))
        return words_result
    except:
        print(Sentence)
        return([])

In [4]:
Sentence = "▪️nwot Look cool and on trend in this striped midi bodycon dress. It's easy to style up or down- pair with black heels (or sandals for a more casual look) and you're good to go! ❤️NEW! All purchases over [rm] (excluding shipping) will get a sweet surprise gift!❤️ Stretchy lace thong style panties brand new with tags."
print(NLP_process(Sentence))

['nwot', 'look', 'cool', 'trend', 'striped', 'midi', 'bodycon', 'dress', 's', 'easy', 'style', 'down', 'pair', 'black', 'heel', 'sandal', 'casual', 'look', 're', 'good', 'go', 'new', 'purchase', 'rm', 'excluding', 'shipping', 'get', 'sweet', 'surprise', 'gift', 'stretchy', 'lace', 'thong', 'style', 'panty', 'brand', 'new', 'tag']


In [5]:
Train = pd.read_csv("train.tsv",delimiter = '\t')
Test = pd.read_csv("test.tsv",delimiter = '\t')

In [6]:
Train.head(1)

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet


In [7]:
Train_Meta = pd.concat([Train.train_id,Train.category_name,Train.item_description,Train.price],axis =1)
Test_Meta = pd.concat([Test.test_id, Test.category_name ,Test.item_description],axis = 1)

In [8]:
Train_Meta.head(1)

,train_id,category_name,item_description,price
0,0,Men/Tops/T-shirts,No description yet,10.0


In [9]:
Train_Meta = Train_Meta.rename(columns={'train_id':'id'})
Test_Meta = Test_Meta.rename(columns = {'test_id':'id'})
Train_Meta['if_Train'] = 1
Test_Meta['if_Train'] = 0
Train_and_Test = pd.concat(
    [Train_Meta.drop(['price'],axis=1),Test_Meta],axis = 0)

In [10]:
Train_and_Test.category_name = Train_and_Test.category_name.astype('category')
Category_Dict = dict( enumerate(Train_and_Test['category_name'].cat.categories))
Train_and_Test.category_name = Train_and_Test.category_name.cat.codes
Coded_Train = Train_and_Test.loc[Train_and_Test['if_Train'] == 1]
Coded_Test = Train_and_Test.loc[Train_and_Test['if_Train'] == 0]

In [11]:
Coded_Train = Coded_Train.drop(['if_Train'],axis = 1)
Coded_Test = Coded_Test.drop(['if_Train'],axis = 1)
Coded_Train['price'] = Train_Meta.price

In [12]:
del Train
del Test
del Train_Meta
del Test_Meta
del Train_and_Test

In [13]:
Coded_Train.head()

,id,category_name,item_description,price
0,0,829,No description yet,10.0
1,1,86,This keyboard is in great condition and works ...,52.0
2,2,1277,Adorable top with a hint of lace and a key hol...,10.0
3,3,503,New with tags. Leather horses. Retail for [rm]...,35.0
4,4,1204,Complete with certificate of authenticity,44.0


In [26]:
Coded_Train.item_description = Coded_Train.item_description.apply(NLP_process)

nan
nan
nan
nan


In [33]:
Coded_Train.head(1)

,id,category_name,item_description,price
0,0,829,"[description, yet]",10.0


In [49]:
Dict_Word_Value = {}
p_mean = Coded_Train.price.mean()
p_STD = Coded_Train.price.std()

In [50]:
def updateDicrWordValue(DF):
    x_std = (DF['price']- p_mean) / p_STD
    for word in DF['item_description']:        
        if Dict_Word_Value.get(word) == None:
            Dict_Word_Value[word] = [x_std]
        else:
            Dict_Word_Value[word].append(x_std)

In [51]:
Coded_Train.apply(updateDicrWordValue,axis=1)

0          None
1          None
2          None
3          None
4          None
5          None
6          None
7          None
8          None
9          None
10         None
11         None
12         None
13         None
14         None
15         None
16         None
17         None
18         None
19         None
20         None
21         None
22         None
23         None
24         None
25         None
26         None
27         None
28         None
29         None
           ... 
1482505    None
1482506    None
1482507    None
1482508    None
1482509    None
1482510    None
1482511    None
1482512    None
1482513    None
1482514    None
1482515    None
1482516    None
1482517    None
1482518    None
1482519    None
1482520    None
1482521    None
1482522    None
1482523    None
1482524    None
1482525    None
1482526    None
1482527    None
1482528    None
1482529    None
1482530    None
1482531    None
1482532    None
1482533    None
1482534    None
Length: 1482535, dtype: 

In [53]:
import numpy as np
for key in Dict_Word_Value :
    value = np.mean(Dict_Word_Value[key])
    Dict_Word_Value[key] = value

In [54]:
for key in Dict_Word_Value:
    print(Dict_Word_Value[key])
    
    break

-0.111639570967


In [57]:
Category_Dictionary = {}
for key in Category_Dict:
    Category_Dictionary[Category_Dict[key]] = key

In [58]:
import json
with open("Gorbal_Descript_Meta.json",'w') as o_f:
    json.dump(Dict_Word_Value,o_f)
with open("Gorbal_Category_Meta.json",'w') as o_f:
    json.dump(Category_Dictionary,o_f)